In [13]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import cv2
import os
import cv2
import random
import keras.backend as K
from matplotlib import pyplot as plt

In [14]:
#import les directory
DIR_TRAINING='/root/Eve_Projet_Melanome/images/benin_malin/train'
DIR_TESTING='/root/Eve_Projet_Melanome/images/benin_malin/test'

In [15]:
#pour batir notre dataset, nous allons utiliser les dictionnaires
#les classes que l'on tente de predire
CLASSES = os.listdir(DIR_TRAINING)
CLASSES

['malin', 'benin']

In [16]:
training={classe:[ "{}/{}/{}".format(DIR_TRAINING, classe,fichier) for fichier in os.listdir("{}/{}".format(DIR_TRAINING, classe))] for classe in CLASSES}
#for classe in CLASSES ici on recupère les clés
testing={classe:[ "{}/{}/{}".format(DIR_TESTING, classe,fichier) for fichier in os.listdir("{}/{}".format(DIR_TESTING, classe))] for classe in CLASSES}


In [17]:
for classe , images in training.items():
  print(classe,len(images), "images")


malin 2803 images
benin 26034 images


In [18]:
for classess , imagess in testing.items():
  print(classess,len(imagess), "images")

malin 117 images
benin 6508 images


In [19]:
class GenerateurImages(tf.keras.utils.Sequence):

    def __init__(self, dataset,batch_size=8, image_shape=(480, 480) ):
      self.classes= list(dataset.keys())
      self.dataset=[(image, [1 if i==self.classes.index(classe) else 0 for i in range(len(self.classes))])
      for image in dataset[classe]
      for classe in self.classes]
      self.batch_size=batch_size
      self.image_shape=image_shape
      random.shuffle(self.dataset)

    def on_epoch_end(self):
      random.shuffle(self.dataset)

    def __len__(self):
      return len(self.dataset) // self.batch_size

    def __getitem__(self, idx):
        batch= self.dataset[idx*self.batch_size : (idx+1)*self.batch_size]
        X,y=[],[]
        for paire in batch:
          X.append(self.prepare_x(paire[0]))
          y.append(np.asarray(paire[1]).astype(np.float32))
          return np.asarray(X).astype(np.float32), np.asarray(y).astype(np.float32)



    #fonction qui travaille l'image(nom) et retourne une matrice normalisée
    def prepare_x(self, x):
      image= cv2.imread(x, cv2.IMREAD_GRAYSCALE)
      image= cv2.resize(image, [self.image_shape[1],self.image_shape[0]] )
      image= np.asarray(image).astype(np.float32)/ 255.0
      return image

In [20]:
#Creer des instances des generateur
generateur_training= GenerateurImages(training)
generateur_testing= GenerateurImages(testing)


In [21]:
#creer model keras
modelResnet=tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(480,480,1),
    classes=len(CLASSES),
    classifier_activation="softmax"
)


In [22]:
# fonction pour calculer le f1- score
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [23]:
# Définition du callback pour suivre le meilleur score
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    '/root/hassan_projet_melanome/hassan_Traitement_test/sauvegarder/modelEfficientNetV2B2.h5',
    monitor='val_f1',
    mode='max',
    save_best_only=True,    
    verbose=1
)

# Utilisation des poids d'échantillonnage pour le déséquilibre de classes
class_weights = {0: 1, 1: 10}  # Adapter les poids selon le déséquilibre des classes


In [24]:
#Compiler et enrainer
modelResnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy", get_f1])

In [25]:
modelResnet.fit(
    x=generateur_training,
    epochs=2,
    validation_data=generateur_testing,
    callbacks=[checkpoint_callback],
    class_weight=class_weights  # Utiliser les poids d'échantillonnage pour le déséquilibre
)

# Définir le chemin du fichier de sauvegarde
save_path = "/root/Eve_Projet_Melanome/Modeles/modelResnet.h5"

# Enregistrer le modèle
modelResnet.save(save_path)

print("Modèle enregistré avec succès.")

Epoch 1/2


2023-07-05 14:09:49.958088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


6508/6508 [==============================] - ETA: 0s - loss: 1.8611 - accuracy: 0.4922 - get_f1: 0.4922

2023-07-05 16:35:07.638361: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


6508/6508 [==============================] - 9281s 1s/step - loss: 1.8611 - accuracy: 0.4922 - get_f1: 0.4922 - val_loss: 316.2477 - val_accuracy: 0.5071 - val_get_f1: 0.5071
Epoch 2/2
6508/6508 [==============================] - 9393s 1s/step - loss: 1.7178 - accuracy: 0.5089 - get_f1: 0.5089 - val_loss: 1.5306 - val_accuracy: 0.4892 - val_get_f1: 0.4892
Modèle enregistré avec succès.


In [26]:
# Partie 1 - Faites les predictions sur les images
image= cv2.imread("/root/Eve_Projet_Melanome/images/Image de prediction/images.jpg",cv2.IMREAD_GRAYSCALE)
image1= cv2.imread("/root/Eve_Projet_Melanome/images/Image de prediction/Melanoma.jpg",cv2.IMREAD_GRAYSCALE)
image2= cv2.imread("/root/Eve_Projet_Melanome/images/Image de prediction/mole_high_he_fr.jpg",cv2.IMREAD_GRAYSCALE)
#redimensionnement de l'image
#redimensionnement de l'image

#redimensionnement de l'image
ts_res=cv2.resize(image,(480,480))
ts_res1=cv2.resize(image1,(480,480))
ts_res2=cv2.resize(image2,(480,480))
#Normalisation de l'image
ts_norm=np.asarray(ts_res).astype(np.float32)/255.0
ts_norm1=np.asarray(ts_res1).astype(np.float32)/255.0
ts_norm2=np.asarray(ts_res2).astype(np.float32)/255.0
#etendre la dimension de l'image
ts_exp=np.expand_dims(ts_norm, axis=0)
ts_exp1=np.expand_dims(ts_norm1, axis=0)
ts_exp2=np.expand_dims(ts_norm1, axis=0)
#prediction
pred=modelResnet.predict(ts_exp)
pred1=modelResnet.predict(ts_exp1)
pred2=modelResnet.predict(ts_exp1)
cv2_imshow(image)
print("Prediction:",pred[0])
cv2_imshow(image1)
print("Prediction:",pred1[0])
cv2_imshow(image2)
print("Prediction:",pred2[0])

1/1 [==============================] - 0s 346ms/step


NameError: name 'cv2_imshow' is not defined